In [5]:
import numpy as np
from numpy import subtract as min
from numpy import multiply as mult
from numpy import divide as div
from numpy.random import randint as random
import pandas as pd
import matplotlib.pyplot as plt

bacth_size=100
epoches_no=10
x_train,y_train,x_test,y_test=[],[],[],[]
df=pd.read_csv("/content/sample_data/mnist_train_small.csv")
no_rows,no_cols=df.shape
y_temp=df.iloc[:,0]
x_temp=df.iloc[:,1:]

In [ ]:
def create(i):
  idk=np.zeros(10)
  idk[i]=1
  return idk
for i in y_temp:
  y_train.append(create(i))
for i in range(no_rows):
  x_train.append(np.asarray(x_temp.iloc[i]))
print(x_train[0])

In [7]:
def sigmoid(x):
  return 1/(1+np.exp(-x))
def ReLu(x):
  return max(0.1*x,x)
class layer:
  def __init__(self,inputSize,outputSize,activation_type):
    self.weigths=[[random(1,101)/100000 for _ in range(inputSize)] for _ in range(outputSize)]
    self.baises=[random(1,101)/100000  for _ in range(outputSize)]
    self.actvation_type=activation_type
    self.outputSize=outputSize
    self.inputSize=inputSize
    self.activation_run_layer,self.node_run_layer,self.der_activation_layer=[0 for _ in range(outputSize)],[0 for _ in range(outputSize)],[0 for _ in range(outputSize)]
    self.LR=0.05
    self.input=[[]]

  def runLayer(self,inputVec):
    self.input[0]=inputVec # inputing 2d array rather then 1d array
    for index in range(self.outputSize):
      self.node_run_layer[index]=(np.dot(self.weigths[index],inputVec)+self.baises[index])
    if self.actvation_type=='relu' :
      for index in range(self.outputSize):
        self.activation_run_layer[index]=(max(0,self.node_run_layer[index]))
    if self.actvation_type=='sigmoid' :
      for index in range(self.outputSize):
        self.activation_run_layer[index]=sigmoid(self.node_run_layer[index])

  def derivative_activation_layer(self):
    if self.actvation_type == 'relu':
      for i in range (self.outputSize):
        if self.node_run_layer[i] > 0:
          self.der_activation_layer[i]=1
        else:
          self.der_activation_layer[i]=0
    elif self.actvation_type=='sigmoid':
      for i in range(self.outputSize):
        self.der_activation_layer[i]=sigmoid(self.node_run_layer[i])*(1-sigmoid(self.node_run_layer[i]))

  def UpdateParameter(self,derivative_weigths,derivative_baises,bacth_size):
    self.weigths=min(self.weigths , div(mult(self.LR,derivative_weigths),bacth_size))
    self.baises=min(self.baises , div(mult(self.LR,derivative_baises), bacth_size))

def runModel(model,inputVec):
  for i in range(len(model)):
    model[i].runLayer(inputVec)
    inputVec=model[i].activation_run_layer

In [8]:
def loss(model,x_train_single,y_train_single):
  runModel(model,x_train_single)
  return sum(min(y_train_single,model[len(model)-1].activation_run_layer))*sum(min(y_train_single,model[len(model)-1].activation_run_layer))
def Avgcost(model,x,y):
  cost=0
  for i in range(len(x)):
    cost+=loss(model,x[i],y[i])
  return cost/len(x)

In [9]:
def backProp(model,x_train_single,y_train_single,bacth_size):
  len_of_model=len(model)
  runModel(model,x_train_single)
  predicted_output=model[len_of_model-1].activation_run_layer
  expected_output=y_train_single
  dervatie_of_layer,dervatie_of_weigths,dervatie_of_baises=[[0] for i in range(len_of_model)],[[0] for i in range(len_of_model)],[[0] for i in range(len_of_model)]
  # calculating last layer dervatie
  model[len_of_model-1].derivative_activation_layer()
  dervatie_of_layer[len_of_model-1]=[model[len_of_model-1].der_activation_layer * min(expected_output , predicted_output  )  * -2]
  for index in range(len_of_model-2,-1,-1):
    model[index].derivative_activation_layer()
    dervatie_of_layer[index]=[np.sum(mult(model[index].der_activation_layer , model[index+1].weigths ) * np.transpose(dervatie_of_layer[index+1]),axis=0)]
  for index in range(len_of_model):
    dervatie_of_weigths[index]=mult( model[index].input,np.transpose(dervatie_of_layer[index]) )
    model[index].UpdateParameter(dervatie_of_weigths[index],dervatie_of_layer[index][0],bacth_size)
  return model
def trainModel(model,epoches_no,x,y,bacth_size):
  for epoches in range(epoches_no):
    for example_index in range(len(x)):
      model=backProp(model1,x[example_index],y[example_index],bacth_size)
    print(epoches)
  return model

In [10]:
model1=[layer(inputSize=784,outputSize=128,activation_type='relu'),
        layer(inputSize=128,outputSize=10,activation_type='sigmoid')]
runModel(model1,x_train[0])
print(model1[len(model1)-1].activation_run_layer)
model1=trainModel(model1,epoches_no,x_train,y_train,bacth_size)

[0.687558450273595, 0.6875224457339734, 0.6943156780820132, 0.690537261573638, 0.702752975347551, 0.6893163687130905, 0.702588310708659, 0.6889601945567558, 0.7177504498251712, 0.6975176581074493]
0
1
2
3
4
5
6
7
8
9


In [11]:
print(Avgcost(model1,x_train,y_train))
count=0
for i in range(no_rows):
  runModel(model1,x_train[i])
  if model1[len(model1)-1].activation_run_layer.index(max(model1[len(model1)-1].activation_run_layer))==y_temp[i]:
    count+=1
print((count/no_rows)*100)
# for i in range(no_rows):
#   runModel(model1,x_train[i])
#   if model1[len(model1)-1].activation_run_layer.index(max(model1[len(model1)-1].activation_run_layer))!=y_temp[i]:
#     print(model1[len(model1)-1].activation_run_layer.index(max(model1[len(model1)-1].activation_run_layer)),y_temp[i])
#     arr_2d = np.reshape(x_train[i], (28, 28))
#     plt.imshow(arr_2d, cmap='gray')
#     input(":")
#     plt.show()

0.0435357454339221
96.32981649082454


In [ ]:
df=pd.read_csv("/content/sample_data/mnist_test.csv")
no_rows,no_cols=df.shape
y_temp=df.iloc[:,0]
x_temp=df.iloc[:,1:]
for i in y_temp:
  y_test.append(create(i))
for i in range(no_rows):
  x_test.append(np.asarray(x_temp.iloc[i]))
print(x_test[0])

In [13]:
print(Avgcost(model1,x_test,y_test))
count=0
for i in range(no_rows):
  runModel(model1,x_test[i])
  if model1[len(model1)-1].activation_run_layer.index(max(model1[len(model1)-1].activation_run_layer))==y_temp[i]:
    count+=1
print((count/no_rows)*100)

0.05110043817615938
94.96949694969497
